In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from the CRUD module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Update with your username and password for MongoDB
username = "aacuser"
password = "testTest"

# Connect to database via CRUD Module
db = AnimalShelter()

# Fetch initial data from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to avoid issues with ObjectID
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with the correct path to your logo file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Branding elements with logo and unique identifier
branding = html.Div([
    html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '150px'}),
        href='https://www.snhu.edu',
        target='_blank'
    ),
    html.P("Developed by Jarrod Helmers", style={'fontSize': '18px', 'fontWeight': 'bold'})
])

# Interactive filter options
filter_options = html.Div([
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'block'}
    )
])

# Dashboard layout
app.layout = html.Div([
    branding,
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    filter_options,
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Adding pagination for better usability
        sort_action='native'  # Adding sorting feature
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    # Fetch filtered data from MongoDB
    filtered_data = db.read(query)
    return pd.DataFrame.from_records(filtered_data).drop(columns=['_id']).to_dict('records')

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Run the server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:27023/
